In [26]:
#import necessary tools + libraries
%pip install pandas
import requests
from bs4 import BeautifulSoup
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd

3599.87s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 11.3 MB 7.6 MB/s eta 0:00:01
     |████████████████████████████████| 345 kB 30.7 MB/s eta 0:00:01
  Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
You should consider upgrading via the '/usr/local/bin/python3.10 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [27]:
#scrapes the first four pages of the best buy website 
def scrape_best_buy_laptops():
    urlDict = []
    headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"} 
    urls = ['https://www.bestbuy.com/site/searchpage.jsp?id=pcat17071&st=laptops+computers+on+sale', 
            'https://www.bestbuy.com/site/searchpage.jsp?cp=2&id=pcat17071&st=laptops+computers+on+sale', 
            'https://www.bestbuy.com/site/searchpage.jsp?cp=3&id=pcat17071&st=laptops+computers+on+sale', 
            'https://www.bestbuy.com/site/searchpage.jsp?cp=4&id=pcat17071&st=laptops+computers+on+sale']
    
    laptops = [] #laptop_data is a dictionary with key-value pairs (key = name)
    for url in urls: #iterate through all 4 pages 
        response = requests.get(url = url, headers = headers)
        soup = BeautifulSoup(response.content, 'html.parser') # If this line causes an error, run 'pip install html5lib' or install html5lib
        # print(soup.prettify()) 
        for row in soup.findAll('h4', attrs = {'class':'sku-title'}):  # find the names of the laptops
            laptops.append(row.a.text) #add the key-value pairs (ex. name-"lenovo thinkpad...") to the dictionary

    brand_list=[]
    mem_list = []
    processor_list = []
    screen_list = []
    touch_list = []

    laptop_dictionary={"brand":brand_list, "memorySize":mem_list, "processor":processor_list, 
                       "screen":screen_list, "touch": touch_list}

    for i in laptops:
        split = i.split('- ')
        if(len(split)<5):
            split = i.split(' -')

        #print(len(split))
        split_2 = split[1].split('"')
        #for j in split:
         #   print(j)
        #correct up to this point 

        brand_list.append(split[0])
        if  len(split) < 5:
            mem_list.append("N/A")
        else:
            mem_list.append(split[3])
        processor_list.append(split[2])
        screen_list.append(split_2[0])
        if len(split_2)>1:
            if "touch" in split_2[1].lower():
                touch_list.append("true")
            else:
               touch_list.append("false")     
        else:
            touch_list.append("false")

    

    for key, value_list in laptop_dictionary.items():
        print(f"Elements for {key}:")
        for element in value_list:
                print(element)
        print("--------")
    return laptop_dictionary

#print(len(laptop_data))
#print(type(laptop_data[0]))
laptop_data = scrape_best_buy_laptops() 
print(laptop_data)



Elements for brand:
Dell 
HP 
HP 
HP 
HP 
ASUS 
Acer 
HP 
Lenovo 
Lenovo 
Microsoft 
Lenovo 
Dell 
Acer 
HP 
Lenovo
Microsoft 
Lenovo 
ASUS 
Lenovo Slim 7i 16" WQXGA Touchscreen Laptop 
Apple 
Lenovo 
HP 
Lenovo 
Dell 
Dell 
LG 
HP 
HP 
Dell 
Lenovo Ideapad 1 15.6" Laptop 
HP 
Lenovo 
MacBook Pro 13.3" Laptop 
Samsung 
Lenovo
Lenovo 
HP 
HP 
Dell 
HP 
HP 
Lenovo
ASUS 
HP 
Dell 
Dell 
ASUS 
HP 
HP 
MacBook Pro 13.3" Laptop 
HP 
Dell 
ASUS 
ASUS
Lenovo 
Lenovo 
Microsoft 
ASUS 
Dell 
Lenovo Ideapad 1 14" Laptop 
Acer Aspire 1
Acer
ASUS 
Lenovo 
HP 
Lenovo 
Lenovo IdeaPad 3 15IJL6 15.6" Chromebook Intel Celeron N4500 4GB Ram 64GB eMMC Chrome OS
HP 
Dell 
Dell 
Lenovo 
--------
Elements for memorySize:
16GB Memory 
16GB Memory 
16GB Memory 
16GB Memory 
4GB Memory 
512GB SSD 
AMD Ryzen 5 7520U 
4GB Memory 
Intel Core i7-1355U with 16GB Memory 
512GB SSD 
Intel Evo Platform Core i7 with 16GB Memory 
Intel Core i5-1335U with 16GB Memory 
16GB Memory 
16GB DDR4
8GB Memory 
 512GB SSD
Intel Co

In [33]:
import re

def scrapeBestBuyLaptopsv2():
    headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"} 
    urls = ['https://www.bestbuy.com/site/searchpage.jsp?id=pcat17071&st=laptops+computers+on+sale', 
            'https://www.bestbuy.com/site/searchpage.jsp?cp=2&id=pcat17071&st=laptops+computers+on+sale', 
            'https://www.bestbuy.com/site/searchpage.jsp?cp=3&id=pcat17071&st=laptops+computers+on+sale', 
            'https://www.bestbuy.com/site/searchpage.jsp?cp=4&id=pcat17071&st=laptops+computers+on+sale']
    
    laptop_strings = [] #laptop_strings is a list of each product's main listing
    for url in urls: #iterate through all 4 pages 
        response = requests.get(url = url, headers = headers)
        soup = BeautifulSoup(response.content, 'html.parser') # If this line causes an error, run 'pip install html5lib' or install html5lib
        # print(soup.prettify()) 
        for row in soup.findAll('h4', attrs = {'class':'sku-title'}):  # find the names of the laptops
            laptop_strings.append(row.a.text) #add info to dictionary
            product_to_data_map

    laptops_info = extract_laptop_info(laptop_strings)
    print("laptop info: " + str(laptops_info))
    print("strings: " + str(laptop_strings))

def extract_laptop_info(laptop_strings):
    laptop_info_list = []
    
    for string in laptop_strings:
        # Initialize dictionary with default values
        laptop_dict = {'brand': "n/a", 'memorySize': "n/a", 'processor': "n/a", 'screen': "n/a", 'touch': "n/a"}
        
        # Extract brand
        brand_match = re.search(r"^\w+", string)
        if brand_match:
            laptop_dict['brand'] = brand_match.group(0)
        
        # Extract memory size
        memory_match = re.search(r"(\d+GB) Memory", string)
        if memory_match:
            laptop_dict['memorySize'] = memory_match.group(1)
        
        # Extract processor
        processor_match = re.search(r"Intel [^\-]+|AMD [^\-]+|M1|M2|M3", string)
        if processor_match:
            laptop_dict['processor'] = processor_match.group(0)
        
        
        # Extract screen size
        screen_match = re.search(r"(\d+(?:\.\d+)?)\"", string)
        if screen_match:
            laptop_dict['screen'] = screen_match.group(1) + '"'
        
        # Extract touchscreen data
        touch_match = re.search(r"2-in-1|Touchscreen|touch", string, re.IGNORECASE)
        if touch_match:
            laptop_dict['touch'] = 'Yes'
        else:
            laptop_dict['touch'] = 'No'
        
        # Append the dictionary to the list
        laptop_info_list.append(laptop_dict)
    
    return laptop_info_list

laptops_info = scrapeBestBuyLaptopsv2()


laptop info: [{'brand': 'Dell', 'memorySize': '16GB', 'processor': 'AMD Ryzen 5 7530U ', 'screen': '16.0"', 'touch': 'Yes'}, {'brand': 'HP', 'memorySize': '16GB', 'processor': 'Intel Core i7 ', 'screen': '15.6"', 'touch': 'Yes'}, {'brand': 'HP', 'memorySize': '16GB', 'processor': 'AMD Ryzen 5 7520U ', 'screen': '15.6"', 'touch': 'No'}, {'brand': 'Acer', 'memorySize': 'n/a', 'processor': 'AMD Ryzen 5 7520U ', 'screen': '15.6"', 'touch': 'Yes'}, {'brand': 'HP', 'memorySize': '16GB', 'processor': 'Intel Core i7 ', 'screen': '15.6"', 'touch': 'Yes'}, {'brand': 'HP', 'memorySize': '4GB', 'processor': 'Intel Celeron ', 'screen': '14"', 'touch': 'No'}, {'brand': 'ASUS', 'memorySize': '12GB', 'processor': 'AMD Ryzen 7 5800HS with 12GB Memory ', 'screen': '16"', 'touch': 'No'}, {'brand': 'HP', 'memorySize': '4GB', 'processor': 'Intel Celeron ', 'screen': '14"', 'touch': 'No'}, {'brand': 'Lenovo', 'memorySize': '16GB', 'processor': 'Intel Evo Platform ', 'screen': '14"', 'touch': 'Yes'}, {'brand

In [29]:
SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True
%pip install sklearn
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import OneHotEncoder
import numpy as np


def memory_size_to_num(memory_size):
    return int(memory_size.replace('GB', ''))

def knnFunction(laptops_info, memorySize, processor, brand, screen,touch):
    brands = [laptop['brand'] for laptop in laptops_info] + [brand]
    processors = [laptop['processor'] for laptop in laptops_info] + [processor]
    screens = [laptop['screen'] for laptop in laptops_info] + [screen]
    touches = [laptop['touch'] for laptop in laptops_info] + [touch]

    encoder = OneHotEncoder()

    # One-hot encode the categorical data
    encoded_brands = encoder.fit_transform(np.array(brands).reshape(-1, 1)).toarray()
    encoded_processors = encoder.fit_transform(np.array(processors).reshape(-1, 1)).toarray()
    encoded_screens = encoder.fit_transform(np.array(screens).reshape(-1, 1)).toarray()
    encoded_touches = encoder.fit_transform(np.array(touches).reshape(-1, 1)).toarray()

    # Convert laptops_info to numerical data
    laptops_numerical = []
    for i, laptop in enumerate(laptops_info):
        laptops_numerical.append(np.concatenate([
            [memory_size_to_num(laptop['memorySize'])],
            encoded_brands[i],
            encoded_processors[i],
            encoded_screens[i],
            encoded_touches[i]
        ]))

    # Convert user preferences to numerical data
    user_preferences = np.concatenate([
        [memory_size_to_num(memorySize)],
        encoded_brands[-1],  # The last entry corresponds to the user's preference
        encoded_processors[-1],
        encoded_screens[-1],
        encoded_touches[-1]
    ])

    # Use NearestNeighbors to find the top 3 laptops that match user preferences
    nbrs = NearestNeighbors(n_neighbors=3, algorithm='auto').fit(laptops_numerical)
    distances, indices = nbrs.kneighbors([user_preferences])

    # Get the top 3 matching laptops
    top_3_laptops = [laptops_info[index] for index in indices[0]]

    print(top_3_laptops)

laptops_info = scrapeBestBuyLaptopsv2()
knnFunction(laptops_info, memorySize, processor, brand, screen,touch)

3650.66s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3.10 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


NameError: name 'memorySize' is not defined